In [28]:
import matplotlib.pyplot as plt
import numpy as np
import math

from scipy.integrate import odeint

from bokeh.io import output_notebook
import bokeh.plotting as plt1

from bokeh.palettes import Category10_10
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d
from bokeh.palettes import inferno#select a palette
import itertools# itertools handles the cycling 

from bokeh.palettes import Category10
import itertools

output_notebook()
# Set random seed (for reproducibility)
np.random.seed(1000)

# Start and end time (in milliseconds)
tmin = 0.0
tmax = 50.0

# Average potassium channel conductance per unit area (mS/cm^2)
gK = 36.0

# Average sodoum channel conductance per unit area (mS/cm^2)
gNa = 120.0

# Average leak channel conductance per unit area (mS/cm^2)
gL = 0.3

# Membrane capacitance per unit area (uF/cm^2)
Cm = 1.0

# Potassium potential (mV)
VK = -12.0

# Sodium potential (mV)
VNa = 115.0

# Leak potential (mV)
Vl = 10.613

gkzero = 0.24

# Depolarization (m.mho/cm^2)
gkinf = 15.30

#msec
Tau = 1.7


def gkth(tt, gkinf, n):
    temp1 = gkinf ** (0.25)
    temp2 = (gkinf ** 0.25 - gkzero ** 0.25)
    t = np.linspace(0, tt, n)
    temp3 = (temp1 - temp2 * (np.exp(-t/Tau))) ** 4
    return t, temp3
 

# Time values
T = np.linspace(tmin, tmax, 10000)

# Potassium ion-channel rate functions

def alpha_n(Vm):
    return (0.01 * (10.0 - Vm)) / (np.exp(1.0 - (0.1 * Vm)) - 1.0)

def beta_n(Vm):
    return 0.125 * np.exp(-Vm / 80.0)

# Sodium ion-channel rate functions

def alpha_m(Vm):
    return (0.1 * (25.0 - Vm)) / (np.exp(2.5 - (0.1 * Vm)) - 1.0)

def beta_m(Vm):
    return 4.0 * np.exp(-Vm / 18.0)

def alpha_h(Vm):
    return 0.07 * np.exp(-Vm / 20.0)

def beta_h(Vm):
    return 1.0 / (np.exp(3.0 - (0.1 * Vm)) + 1.0)
  
# n, m, and h steady-state values

def n_inf(Vm=0.0):
    return alpha_n(Vm) / (alpha_n(Vm) + beta_n(Vm))

def m_inf(Vm=0.0):
    return alpha_m(Vm) / (alpha_m(Vm) + beta_m(Vm))

def h_inf(Vm=0.0):
    return alpha_h(Vm) / (alpha_h(Vm) + beta_h(Vm))
  
    
#displacement voltage
def alphacol(Vdis):
    nr = 0.01 * (Vdis + 10)
    dr = np.exp((Vdis+10)/10) - 1
    return nr/dr

def betacol(Vdis):
    return 0.125 * np.exp(Vdis/80)

def ninf(a, b, c):
    v = np.linspace(a, b, c)
    temp = alphacol(v) / (alphacol(v) + betacol(v))
    return v, temp

def gna_t(a, b, g1, taum, tauh):
    t = np.linspace(0, a, b)
    temp = g1 * ((1 - np.exp(-t/taum)) ** 3) * np.exp(-t/tauh)
    return t, temp

def k_rate_alpha(a, b, c):
    v = np.linspace(a, b, c)
    nr = 0.01 * (v + 10)
    dr = np.exp((v+10)/10) - 1
    temp = nr / dr
    return v, temp

def k_rate_beta(a, b, c):
    v = np.linspace(a, b, c)
    temp = 0.125 * np.exp(v/80)
    return v, temp


def na_rate_alpha(a, b, c):
    v = np.linspace(a, b, c)
    nr = 0.1 * (v + 25)
    dr = np.exp((v+25)/10) - 1
    temp = nr / dr
    return v, temp

def na_rate_beta(a, b, c):
    v = np.linspace(a, b, c)
    temp = 4 * np.exp(v/18)
    return v, temp
 

    
def rate_alphanew(v):
    nr = 0.1 * (v + 25)
    dr = np.exp((v+25)/10) - 1
    temp = nr / dr
    return temp

def rate_betanew(v):
    temp = 4 * np.exp(v/18)
    return temp

def minf(a, b, c):
    v = np.linspace(a, b, c)
    temp = rate_alphanew(v) / (rate_alphanew(v) + rate_betanew(v))
    return v, temp

def alpha_hnew(a, b, c):
    v = np.linspace(a, b, c)
    temp = 0.07 * np.exp(v/20)
    return v, temp

def beta_hnew(a, b, c):
    v = np.linspace(a, b, c)
    nr = 1
    dr = np.exp((v+30)/10) + 1
    temp = nr / dr
    return v, temp
   
def alpha_hhelp(v):
    temp = 0.07 * np.exp(v/20)
    return temp

def beta_hhelp(v):
    nr = 1
    dr = np.exp((v+30)/10) + 1
    temp = nr / dr
    return temp
    
def hinf(a, b, c):
    v = np.linspace(a, b, c)
    temp = alpha_hhelp(v)/(alpha_hhelp(v) + beta_hhelp(v))
    return v, temp
    
# Input stimulus
def Id(t):
    if 0.0 < t < 1.0:
        return 150.0
    elif 10.0 < t < 11.0:
        return 50.0
    return 0.0
  
# Compute derivatives
def compute_derivatives(y, t0):
    dy = np.zeros((4,))
    
    Vm = y[0]
    n = y[1]
    m = y[2]
    h = y[3]
    
    # dVm/dt
    GK = (gK / Cm) * np.power(n, 4.0)
    GNa = (gNa / Cm) * np.power(m, 3.0) * h
    GL = gL / Cm
    
    dy[0] = (Id(t0) / Cm) - (GK * (Vm - VK)) - (GNa * (Vm - VNa)) - (GL * (Vm - Vl))
    
    # dn/dt
    dy[1] = (alpha_n(Vm) * (1.0 - n)) - (beta_n(Vm) * n)
    
    # dm/dt
    dy[2] = (alpha_m(Vm) * (1.0 - m)) - (beta_m(Vm) * m)
    
    # dh/dt
    dy[3] = (alpha_h(Vm) * (1.0 - h)) - (beta_h(Vm) * h)
    
    return dy

def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s))
    return x.translate(res)
  
# State (Vm, n, m, h)
Y = np.array([0.0, n_inf(), m_inf(), h_inf()])

# Solve ODE system
# Vy = (Vm[t0:tmax], n[t0:tmax], m[t0:tmax], h[t0:tmax])
Vy = odeint(compute_derivatives, Y, T)

In [29]:
def color_gen():
    for c in itertools.cycle(Category10[10]):
        yield c
color = color_gen()

In [30]:
fig1 = plt1.figure(x_axis_label="time (msec)", y_axis_label="Potassium conductance (m.mho/cm^2)")
gkinf = [20.70, 20.0, 18.6, 17.0, 15.30, 13.27, 10.29, 8.62, 6.84, 5.0, 1.47, 0.98]
depolV = [-109, -100, -88, -76, -63, -51, -38, -32, -26, -19, -10, -6]
interval = 100
for dvol, gginf, c in zip(depolV, gkinf, color):
    x, y = gkth(12, gginf, interval)
    interval += 5
    labell = 'depolarization value ' + str(gginf) 
    fig1.line(x, y, color=c, legend_label='diplarization = {} mV'.format(dvol))
fig1.legend.location='top_left'
plt1.show(fig1)

In [31]:
fig2 = figure(x_axis_label='V(mV)', y_axis_label='Rate constant (msec^-1)')
v, an = k_rate_alpha(-110, 18, 12);
label_a = ('α{}'.format(get_sub('n')))
label_b = ('β{}'.format(get_sub('n')))
fig2.line(v,an,color='blue',legend_label=label_a)
v, bn = k_rate_beta(-110, 50, 12);
fig2.line(v,bn,color='red', legend_label=label_b)
plt1.show(fig2)

In [32]:
label_y = ('n{}'.format(get_sub('∞')))
fig3=figure(x_axis_label="V (mV)", y_axis_label=label_y)
v, n = ninf(-110, 60, 10)
fig3.line(v, n)
plt1.show(fig3)

In [33]:
fig4 = figure(x_axis_label='Time (ms)', y_axis_label='Sodium conductance m.mho/cm^2')
g1_ar = [40.3, 42.6, 46.8, 39.5, 38.2, 30.7, 20.0, 15.3, 7.9, 0.4, 0.22, 0.2]
taum_ar = [0.14, 0.16, 0.2, 0.189, 0.252, 0.318, 0.382, 0.520, 0.6, 0.14, 0.14, 0.14]
tauh_ah = [0.67, 0.67, 0.67, 0.84, 0.84, 1.06, 1.27, 1.33, 1.5, 2.3, 5.52, 6.73]

for dvol, g11, taumm, tauhh, c in zip(depolV, g1_ar, taum_ar, tauh_ah, color):
    v, n = gna_t(12, 12, g11, taumm, tauhh)
    fig4.line(v,n, color=c, legend_label='diplarization = {} mV'.format(dvol))
plt1.show(fig4)

In [34]:
fig5 = figure(x_axis_label='V(mV)', y_axis_label='Rate constant (msec^-1)')
label_a = ('α{}'.format(get_sub('m')))
label_b = ('β{}'.format(get_sub('m')))
v, am = na_rate_alpha(-110, 10, 12);
fig5.line(v,am,color='blue', legend_label=label_a)
v, bm = na_rate_beta(-110, 10, 12);
fig5.line(v,bm,color='red', legend_label=label_b)
plt1.show(fig5)

In [35]:
label_y = ('m{}'.format(get_sub('∞')))
fig6 = figure(x_axis_label='V(mV)', y_axis_label=label_y)
v, m = minf(-110, 10, 12)
fig6.line(v, m)
plt1.show(fig6)

In [36]:
fig7 = figure(x_axis_label='V(mV)', y_axis_label='Rate constant (msec^-1)')
label_a = ('α{}'.format(get_sub('h')))
label_b = ('β{}'.format(get_sub('h')))
v, a = alpha_hnew(-110, 30, 14)
fig7.line(v,a,color='red',legend_label=label_a)
v, b = beta_hnew(-110, 30, 14)
fig7.line(v,b,color='blue',legend_label=label_b)
plt1.show(fig7)

In [37]:
label_y = ('h{}'.format(get_sub('∞')))
fig8 = plt1.figure(x_axis_label="V (mV)", y_axis_label=label_y)
v, h = hinf(-100, 50, 15)
fig8.line(v, h)
plt1.show(fig8)

In [38]:
## Importing neuron packages

In [39]:
pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:

import neuron
from neuron import h
from neuron import h, rxd
from neuron.units import ms, mV

#### Making new section "soma"

In [41]:

soma = h.Section(name="soma")
soma.L

#setting soma attributes
soma.L = 20
soma.diam = 20

dir(soma)
import textwrap
help(soma.connect)
?soma.connect

# inserting "hh" i.e. hodgkin-huxley model in the soma 
soma.insert("hh")


mech = soma(0.5).hh
iclamp = h.IClamp(soma(0.5))


iclamp.delay = 2
iclamp.dur = 0.1
iclamp.amp = 0.9


v = h.Vector().record(soma(0.5)._ref_v) 
t = h.Vector().record(h._ref_t) 

h.load_file("stdrun.hoc")
h.finitialize(-65 * mV)
h.continuerun(40 * ms)
from bokeh.io import output_notebook
import bokeh.plotting as plt

output_notebook()

Help on built-in function connect:

connect(...) method of nrn.Section instance
    childSection.connect(parentSection, [parentX], [childEnd]) or
    childSection.connect(parentSegment, [childEnd])



In [42]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Potential (mV)")
f.line(t, v, line_width=2)
plt.show(f)

In [43]:
import neuron

from neuron import h
from neuron import h, rxd
from neuron.units import ms, mV
soma = h.Section(name="soma")

soma.L

soma.L = 20
soma.diam = 20

dir(soma)

import textwrap


help(soma.connect)
?soma.connect
soma.insert("hh")


mech = soma(0.5).hh

iclamp = h.IClamp(soma(0.5))


iclamp.delay = 10
iclamp.dur = 0.8
iclamp.amp = 0.9


v = h.Vector().record(soma(0.5)._ref_v)  
t = h.Vector().record(h._ref_t)  

h.load_file("stdrun.hoc")
h.finitialize(-65 * mV)
h.continuerun(40 * ms)
from bokeh.io import output_notebook
import bokeh.plotting as plt

output_notebook()

Help on built-in function connect:

connect(...) method of nrn.Section instance
    childSection.connect(parentSection, [parentX], [childEnd]) or
    childSection.connect(parentSegment, [childEnd])



In [44]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Potential (mV)")
f.line(t, v, line_width=2)
plt.show(f)

In [45]:
import neuron

from neuron import h
from neuron import h, rxd
from neuron.units import ms, mV
soma = h.Section(name="soma")

soma.L

soma.L = 20
soma.diam = 20

dir(soma)

import textwrap


help(soma.connect)
?soma.connect
soma.insert("hh")


mech = soma(0.5).hh

iclamp = h.IClamp(soma(0.5))


iclamp.delay = 10
iclamp.dur = 10
iclamp.amp = 0.9


v = h.Vector().record(soma(0.5)._ref_v)  
t = h.Vector().record(h._ref_t)  

h.load_file("stdrun.hoc")
h.finitialize(-65 * mV)
h.continuerun(40 * ms)
from bokeh.io import output_notebook
import bokeh.plotting as plt

output_notebook()

Help on built-in function connect:

connect(...) method of nrn.Section instance
    childSection.connect(parentSection, [parentX], [childEnd]) or
    childSection.connect(parentSegment, [childEnd])



In [46]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Potential (mV)")
f.line(t, v, line_width=2)
plt.show(f)

In [47]:
import neuron

from neuron import h
from neuron import h, rxd
from neuron.units import ms, mV
soma = h.Section(name="soma")

soma.L

soma.L = 20
soma.diam = 20

dir(soma)

import textwrap


help(soma.connect)
?soma.connect
soma.insert("hh")


mech = soma(0.5).hh

iclamp = h.IClamp(soma(0.5))


iclamp.delay = 10
iclamp.dur = 50
iclamp.amp = 0.9


v = h.Vector().record(soma(0.5)._ref_v)  
t = h.Vector().record(h._ref_t)  

h.load_file("stdrun.hoc")
h.finitialize(-65 * mV)
h.continuerun(40 * ms)
from bokeh.io import output_notebook
import bokeh.plotting as plt

output_notebook()

Help on built-in function connect:

connect(...) method of nrn.Section instance
    childSection.connect(parentSection, [parentX], [childEnd]) or
    childSection.connect(parentSegment, [childEnd])



In [48]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Potential (mV)")
f.line(t, v, line_width=2)
plt.show(f)

### Making new sections

In [49]:
from neuron import h


sec1 = h.Section(name='s1')
sec2 = h.Section(name='s2')
sec3 = h.Section(name='s3')

for sec in [sec1, sec2, sec3]:
    sec.insert('hh')
    sec.L = sec.diam = 3

    
# setting attributes
c1 = h.IClamp(sec1(0.5))
c3 = h.VClamp(sec3(0.5))
c1.dur = 0.1
c1.amp = 0.3
c3.dur[0] = 1

# record an action potential
ap = h.Vector()
ap.record(sec1(0.5)._ref_v)
h.finitialize(-65)
while h.t < 1:
    h.fadvance()

apc = ap.c() 
ap.play_remove()
apc.play(c3._ref_amp[0], h.dt)
h.finitialize(-65)


# cache vectors for membrane potential and current
volt1 = []
volt2 = []
time = []
curr = []

# storing the dyanamic values during simulation
while h.t < 0.4:
    time.append(h.t)
    volt1.append(sec3.v)
    curr.append(c3.i)
    h.fadvance()
    
# importing bokeh
from bokeh.io import output_notebook
import bokeh.plotting as plt
output_notebook()




In [50]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Potential (mV)")
f.line(time, volt1, line_width=2)
plt.show(f)

In [51]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="Membrane Current (nA)")
f.line(time, curr, line_width=2)
plt.show(f)